In [70]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import librosa
import numpy as np
import librosa.display
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
from server.database_wrapper import PostgresqlWrapper
from server.utils import Util
from server.feature_extractor import FeatureAggregator
import xgboost as xgb
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import StratifiedKFold
from numpy import binary_repr
from sklearn.linear_model import LogisticRegression

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Импортим данные

In [2]:
data = pd.DataFrame.from_csv("extracted_data_new.csv", encoding="utf-8").values
X = np.array(data[:, :-1], dtype = float)
genre_list = data[:, -1]

## Определяем модель

In [ ]:
rfc = RandomForestClassifier()
svc = SVC(probability=True)
skf = StratifiedKFold(n_splits=3)
lrg = LogisticRegression(C=1.0, fit_intercept=True, penalty='l2', multi_class = 'multinomial')

param_grid = { 
    'n_estimators': [ 70, 150, 370],
    'max_features': ['log2'],
    'max_depth' : [10,15, 20],
    'criterion' :['gini']
}

param_grid_lgr = {
    "C": np.logspace(0,2,num=5),
    "penalty" : ['l2']
}

grid_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv=skf, n_jobs = -1)
grid_lrg = GridSearchCV(lrg, param_grid=param_grid_lgr, cv=skf, n_jobs = -1, verbose=True)
grid_svc = GridSearchCV(svc, {"C": np.logspace(0,2,num=20), "kernel": ["poly", "rbf", "sigmoid"]}, n_jobs = -1, cv = skf)
eclf = VotingClassifier(estimators=[('svc', grid_svc), ('rfc', grid_rfc)], voting='soft')

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

n = 10

In [ ]:
rfc = RandomForestClassifier()
svc = SVC()
skf = StratifiedKFold(n_splits=3)
lrg = LogisticRegression(C=1.0, fit_intercept=True, penalty='l2', multi_class = 'ovr')

param_grid = { 
'n_estimators': [ 70, 150, 370],
'max_features': ['log2'],
'max_depth' : [10,15, 20],
'criterion' :['gini']
}
param_grid_lgr = {

}

grid_rfc = GridSearchCV(rfc, param_grid=param_grid, cv=skf, n_jobs = -1, verbose=True)
grid_lrg = GridSearchCV(lrg, param_grid=param_grid_lgr, cv=skf, n_jobs = -1, verbose=True)
grid_svc = GridSearchCV(svc, {"C": np.logspace(0,2,num=20), "kernel": ["poly", "rbf", "sigmoid"]}, n_jobs = -1, verbose=True, cv = skf)
eclf = VotingClassifier(estimators=[('svc', grid_svc), ('rfc', grid_rfc)], voting='hard')

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [4]:
def check_model(model, X, y, encoder, n):
    num_genres = len(np.unique(y))
    average = np.zeros((num_genres + 1, 4), dtype = float)
    for rs in tqdm(range(n)):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=rs, stratify=y)
        model.fit(X_train, y_train)
        target = encoder.inverse_transform(np.arange(num_genres))
        df = parse_class_report(classification_report(y_test, model.predict(X_test), target_names=target))
        average += df.values
    df.iloc[:,:] = np.round(average / n, 3)
    return df

def parse_class_report(class_rep_str):
    lines = class_rep_str.split('\n')
    list_splitted = list()
    for line in lines:
        splitted_line = np.array(line.split(' '))
        mask_non_empty = np.array([word != '' for word in line.split(' ')])
        res = splitted_line[mask_non_empty]
        if len(res) != 0:
            list_splitted.append(np.array(res, dtype = object))

    first = list(list_splitted[0])
    first.insert(0, 'score')
    list_splitted[0] = np.array(first, dtype = object)
    list_splitted[-1] = list_splitted[-1][2:]
    list_splitted[-1][0] = 'avg / total'
    
    df = pd.DataFrame(list_splitted)
    df.columns = list(df.iloc[0])
    return df.iloc[1:].set_index('score').convert_objects(convert_numeric=True)

 ## Модель из 6 жанров

In [80]:
mask6 = (genre_list != 'country') & (genre_list != 'blues') &\
        (genre_list != 'disco') & (genre_list != 'reggae')

In [6]:
X6 = X_scaled[mask6]
encoder6 = LabelEncoder()
y6 = encoder6.fit_transform(genre_list[mask6])

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X6, y6, test_size=0.2, random_state=5, stratify=y6)
eclf.fit(X_train, y_train)
target = encoder6.inverse_transform(np.arange(6))
print(classification_report(y_test, eclf.predict(X_test), target_names=target))

             precision    recall  f1-score   support

  classical       0.95      1.00      0.98        20
     hiphop       0.75      0.75      0.75        20
       jazz       1.00      0.95      0.97        20
      metal       0.80      0.80      0.80        20
        pop       0.71      0.85      0.77        20
       rock       0.75      0.60      0.67        20

avg / total       0.83      0.82      0.82       120



/Users/anton/anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/anton/anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [8]:
import warnings
warnings.filterwarnings('ignore')

check_model(grid_rfc, X6, y6, encoder6, 10)

100%|██████████| 10/10 [01:02<00:00,  6.01s/it]


,precision,recall,f1-score,support
score,,,,
classical,0.872,0.920,0.894,20.0
hiphop,0.804,0.800,0.800,20.0
jazz,0.816,0.780,0.796,20.0
metal,0.852,0.855,0.852,20.0
pop,0.773,0.830,0.794,20.0
rock,0.710,0.625,0.660,20.0
avg / total,0.806,0.800,0.799,120.0


In [12]:
check_model(grid_svc, X6, y6, encoder6, 10)

100%|██████████| 10/10 [00:49<00:00,  5.08s/it]


,precision,recall,f1-score,support
score,,,,
classical,0.917,0.910,0.910,20.0
hiphop,0.807,0.845,0.822,20.0
jazz,0.900,0.840,0.867,20.0
metal,0.823,0.825,0.822,20.0
pop,0.803,0.850,0.823,20.0
rock,0.719,0.670,0.692,20.0
avg / total,0.829,0.823,0.823,120.0


In [13]:
check_model(eclf, X6, y6, encoder6, 10)

100%|██████████| 10/10 [01:58<00:00, 11.58s/it]


,precision,recall,f1-score,support
score,,,,
classical,0.913,0.935,0.922,20.0
hiphop,0.825,0.855,0.836,20.0
jazz,0.910,0.815,0.856,20.0
metal,0.839,0.825,0.830,20.0
pop,0.803,0.855,0.825,20.0
rock,0.716,0.680,0.693,20.0
avg / total,0.835,0.828,0.829,120.0


In [ ]:
grid_lrg

## Модель из 7 жанров

In [9]:
mask7 = (genre_list != 'country') & (genre_list != 'blues') &\
        (genre_list != 'disco')

In [10]:
X7 = X_scaled[mask7]
encoder7 = LabelEncoder()
y7 = encoder7.fit_transform(genre_list[mask7])

In [100]:
X_train, X_test, y_train, y_test = train_test_split(X7, y7, test_size=0.2, random_state=1, stratify=y7)
grid_rfc.fit(X_train, y_train)
target = encoder7.inverse_transform(np.arange(7))
print(classification_report(y_test, grid_rfc.predict(X_test), target_names=target))

,precision,recall,f1-score,support
score,,,,
classical,0.91,1.00,0.95,20
hiphop,0.75,0.75,0.75,20
jazz,0.88,0.70,0.78,20
metal,0.84,0.80,0.82,20
pop,0.81,0.85,0.83,20
reggae,0.70,0.80,0.74,20
rock,0.63,0.60,0.62,20
avg / total,0.79,0.79,0.78,140


In [11]:
check_model(grid_rfc, X7, y7, encoder7, 10)

100%|██████████| 10/10 [01:12<00:00,  7.49s/it]


,precision,recall,f1-score,support
score,,,,
classical,0.839,0.925,0.878,20.0
hiphop,0.820,0.700,0.753,20.0
jazz,0.771,0.760,0.764,20.0
metal,0.809,0.855,0.829,20.0
pop,0.734,0.825,0.774,20.0
reggae,0.698,0.690,0.690,20.0
rock,0.613,0.525,0.564,20.0
avg / total,0.755,0.753,0.750,140.0


In [14]:
check_model(grid_svc, X7, y7, encoder7, 10)

100%|██████████| 10/10 [01:06<00:00,  6.76s/it]


,precision,recall,f1-score,support
score,,,,
classical,0.923,0.930,0.922,20.0
hiphop,0.760,0.820,0.784,20.0
jazz,0.886,0.840,0.861,20.0
metal,0.827,0.840,0.832,20.0
pop,0.810,0.795,0.800,20.0
reggae,0.767,0.760,0.760,20.0
rock,0.713,0.675,0.694,20.0
avg / total,0.812,0.808,0.808,140.0


In [15]:
check_model(eclf, X7, y7, encoder7, 10)

100%|██████████| 10/10 [02:17<00:00, 13.02s/it]


,precision,recall,f1-score,support
score,,,,
classical,0.901,0.930,0.912,20.0
hiphop,0.805,0.830,0.814,20.0
jazz,0.868,0.835,0.849,20.0
metal,0.817,0.850,0.832,20.0
pop,0.816,0.805,0.805,20.0
reggae,0.772,0.755,0.759,20.0
rock,0.697,0.645,0.668,20.0
avg / total,0.811,0.807,0.805,140.0


## Модель из 8 жанров

In [17]:
mask8 = (genre_list != 'disco') & (genre_list != 'reggae')

In [18]:
X8 = X_scaled[mask8]
encoder8 = LabelEncoder()
y8 = encoder8.fit_transform(genre_list[mask8])

In [66]:
X_train, X_test, y_train, y_test = train_test_split(X8, y8, test_size=0.2, random_state=5, stratify=y8)
eclf.fit(X_train, y_train)
target = encoder8.inverse_transform(np.arange(8))
print(classification_report(y_test, eclf.predict(X_test), target_names=target))

Fitting 3 folds for each of 60 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Done  76 tasks      | elapsed:    3.5s
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:    7.9s finished


Fitting 3 folds for each of 9 candidates, totalling 27 fits


[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed:    6.4s finished


             precision    recall  f1-score   support

      blues       0.93      0.70      0.80        20
  classical       0.95      0.90      0.92        20
    country       0.71      0.75      0.73        20
     hiphop       0.74      0.70      0.72        20
       jazz       0.77      0.85      0.81        20
      metal       0.69      0.90      0.78        20
        pop       0.68      0.75      0.71        20
       rock       0.56      0.45      0.50        20

avg / total       0.76      0.75      0.75       160



/Users/anton/anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/anton/anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [19]:
check_model(grid_rfc, X8, y8, encoder8, 10)

100%|██████████| 10/10 [01:10<00:00,  6.87s/it]


,precision,recall,f1-score,support
score,,,,
blues,0.680,0.645,0.657,20.0
classical,0.915,0.920,0.915,20.0
country,0.636,0.635,0.629,20.0
hiphop,0.750,0.795,0.770,20.0
jazz,0.781,0.800,0.789,20.0
metal,0.766,0.820,0.790,20.0
pop,0.745,0.825,0.779,20.0
rock,0.540,0.390,0.442,20.0
avg / total,0.726,0.728,0.721,160.0


In [20]:
check_model(grid_svc, X8, y8, encoder8, 10)

100%|██████████| 10/10 [01:24<00:00,  8.38s/it]


,precision,recall,f1-score,support
score,,,,
blues,0.743,0.790,0.764,20.0
classical,0.956,0.905,0.927,20.0
country,0.718,0.720,0.715,20.0
hiphop,0.801,0.825,0.812,20.0
jazz,0.822,0.825,0.820,20.0
metal,0.813,0.830,0.820,20.0
pop,0.795,0.770,0.774,20.0
rock,0.535,0.490,0.511,20.0
avg / total,0.774,0.768,0.768,160.0


In [21]:
check_model(eclf, X8, y8, encoder8, 10)

100%|██████████| 10/10 [02:42<00:00, 15.34s/it]


,precision,recall,f1-score,support
score,,,,
blues,0.798,0.770,0.781,20.0
classical,0.935,0.925,0.928,20.0
country,0.762,0.705,0.722,20.0
hiphop,0.790,0.840,0.813,20.0
jazz,0.817,0.845,0.826,20.0
metal,0.814,0.840,0.826,20.0
pop,0.790,0.795,0.787,20.0
rock,0.550,0.505,0.518,20.0
avg / total,0.783,0.780,0.774,160.0


## Модель из 9 жанров

In [54]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

In [66]:
def check_model_xgb(X, y, encoder, n):
    num_genres = len(np.unique(y))
    average = np.zeros((num_genres + 1, 4), dtype = float)
    for rs in tqdm(range(n)):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=rs, stratify=y)
        dtrain = xgb.DMatrix(X_train, label=y_train)
        dtest = xgb.DMatrix(X_test, label=y_test)
        param = {'objective': "multi:softmax", "num_class": np.unique(y_train).size}
        evallist = [(dtrain, 'train'), (dtest, 'eval')]
        bst = xgb.train(param, dtrain, evals=evallist)
        target = encoder.inverse_transform(np.arange(num_genres))
        df = parse_class_report(classification_report(y_test, bst.predict(dtest), target_names=target))
        average += df.values
    df.iloc[:,:] = np.round(average / n, 3)
    return df

In [67]:
check_model_xgb(X9, y9, encoder9, 10)

  0%|          | 0/10 [00:00<?, ?it/s]

[0]	train-merror:0.113889	eval-merror:0.444444
[1]	train-merror:0.044444	eval-merror:0.405556
[2]	train-merror:0.025	eval-merror:0.366667
[3]	train-merror:0.011111	eval-merror:0.372222
[4]	train-merror:0.006944	eval-merror:0.366667
[5]	train-merror:0.005556	eval-merror:0.344444
[6]	train-merror:0.004167	eval-merror:0.344444
[7]	train-merror:0.004167	eval-merror:0.327778
[8]	train-merror:0.004167	eval-merror:0.333333
[9]	train-merror:0.004167	eval-merror:0.311111


 10%|█         | 1/10 [00:01<00:11,  1.33s/it]

[0]	train-merror:0.119444	eval-merror:0.466667
[1]	train-merror:0.056944	eval-merror:0.461111
[2]	train-merror:0.034722	eval-merror:0.422222
[3]	train-merror:0.026389	eval-merror:0.4
[4]	train-merror:0.025	eval-merror:0.388889
[5]	train-merror:0.019444	eval-merror:0.383333
[6]	train-merror:0.013889	eval-merror:0.377778
[7]	train-merror:0.009722	eval-merror:0.394444
[8]	train-merror:0.006944	eval-merror:0.383333
[9]	train-merror:0.002778	eval-merror:0.4


 20%|██        | 2/10 [00:02<00:10,  1.31s/it]

...

[0]	train-merror:0.119444	eval-merror:0.45
[1]	train-merror:0.052778	eval-merror:0.394444
[2]	train-merror:0.029167	eval-merror:0.416667
[3]	train-merror:0.015278	eval-merror:0.383333
[4]	train-merror:0.015278	eval-merror:0.366667
[5]	train-merror:0.009722	eval-merror:0.372222
[6]	train-merror:0.002778	eval-merror:0.366667
[7]	train-merror:0.001389	eval-merror:0.388889
[8]	train-merror:0	eval-merror:0.372222
[9]	train-merror:0	eval-merror:0.344444


 90%|█████████ | 9/10 [00:11<00:01,  1.35s/it]

[0]	train-merror:0.106944	eval-merror:0.555556
[1]	train-merror:0.054167	eval-merror:0.5
[2]	train-merror:0.030556	eval-merror:0.427778
[3]	train-merror:0.022222	eval-merror:0.4
[4]	train-merror:0.015278	eval-merror:0.394444
[5]	train-merror:0.0125	eval-merror:0.405556
[6]	train-merror:0.008333	eval-merror:0.388889
[7]	train-merror:0.006944	eval-merror:0.372222
[8]	train-merror:0.006944	eval-merror:0.411111
[9]	train-merror:0.004167	eval-merror:0.394444


100%|██████████| 10/10 [00:13<00:00,  1.44s/it]


,precision,recall,f1-score,support
score,,,,
blues,0.622,0.605,0.606,20.0
classical,0.870,0.875,0.870,20.0
country,0.491,0.495,0.487,20.0
hiphop,0.644,0.575,0.606,20.0
jazz,0.732,0.685,0.705,20.0
metal,0.728,0.790,0.751,20.0
pop,0.749,0.805,0.773,20.0
reggae,0.643,0.665,0.649,20.0
rock,0.458,0.405,0.420,20.0


In [55]:
# Xgboost training
param = {'objective': "multi:softmax", "num_class": np.unique(y_train).size}
evallist = [(dtrain, 'train'), (dtest, 'eval')]
bst = xgb.train(param, dtrain, evals=evallist)

[0]	train-merror:0.131944	eval-merror:0.5
[1]	train-merror:0.069444	eval-merror:0.388889
[2]	train-merror:0.044444	eval-merror:0.383333
[3]	train-merror:0.026389	eval-merror:0.361111
[4]	train-merror:0.009722	eval-merror:0.361111
[5]	train-merror:0.005556	eval-merror:0.344444
[6]	train-merror:0	eval-merror:0.322222
[7]	train-merror:0	eval-merror:0.305556
[8]	train-merror:0	eval-merror:0.311111
[9]	train-merror:0	eval-merror:0.305556


In [65]:
target = encoder9.inverse_transform(np.arange(9))
parse_class_report(classification_report(y_test, bst.predict(dtest), target_names=target))

,precision,recall,f1-score,support
score,,,,
blues,0.68,0.75,0.71,20
classical,0.90,0.90,0.90,20
country,0.69,0.45,0.55,20
hiphop,0.58,0.55,0.56,20
jazz,0.76,0.80,0.78,20
metal,0.76,0.80,0.78,20
pop,0.71,0.85,0.77,20
reggae,0.75,0.75,0.75,20
rock,0.40,0.40,0.40,20


In [22]:
mask9 = (genre_list != 'disco')

In [23]:
X9 = X_scaled[mask9]
encoder9 = LabelEncoder()
y9 = encoder9.fit_transform(genre_list[mask9])

In [49]:
X_train, X_test, y_train, y_test = train_test_split(X9, y9, test_size=0.2, random_state=2, stratify=y9)
eclf.fit(X_train, y_train)
target = encoder9.inverse_transform(np.arange(9))
print(classification_report(y_test, eclf.predict(X_test), target_names=target))

             precision    recall  f1-score   support

      blues       0.86      0.90      0.88        20
  classical       1.00      0.90      0.95        20
    country       0.79      0.55      0.65        20
     hiphop       0.79      0.75      0.77        20
       jazz       0.82      0.90      0.86        20
      metal       0.83      0.75      0.79        20
        pop       0.85      0.85      0.85        20
     reggae       0.65      0.85      0.74        20
       rock       0.41      0.45      0.43        20

avg / total       0.78      0.77      0.77       180



In [50]:
X_train_idx, X_test_idx, y_train_, y_test_ = train_test_split(np.arange(900), y9, test_size=0.2, random_state=2, stratify=y9)

In [51]:
# Как классифицируются роковые песни?
pd.DataFrame(
[list(encoder9.inverse_transform(eclf.predict(X_test)[y_test == 8])[eclf.predict(X_test)[y_test == 8] != 8]),
list(X_test_idx[y_test == 8][eclf.predict(X_test)[y_test == 8] != 8])])

,0,1,2,3,4,5,6,7,8,9,10
0,reggae,blues,country,pop,blues,reggae,country,metal,jazz,blues,country
1,52,19,34,51,64,50,36,74,98,11,06


In [82]:
# Как классифицируются роковые песни?
pd.DataFrame(
[list(encoder9.inverse_transform(eclf.predict(X_test)[y_test == 8])[eclf.predict(X_test)[y_test == 8] != 8]),
list(X_test_idx[y_test == 8][eclf.predict(X_test)[y_test == 8] != 8])])

NotFittedError: This VotingClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this method.

In [ ]:
good = [519 (I want to break ), ]

In [53]:
# Что классификатор считает роком, что в датасете роком не является?
pd.DataFrame(
[list(encoder9.inverse_transform(y_test[(eclf.predict(X_test) == 8) & (y_test != 8) ])),
list(X_test_idx[(eclf.predict(X_test) == 8) & (y_test != 8) ])])

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,country,country,pop,metal,country,metal,blues,metal,metal,classical,country,reggae,hiphop
1,38,43,53,64,82,43,82,58,91,9,80,95,81


In [24]:
check_model(grid_rfc, X9, y9, encoder9, 10)

100%|██████████| 10/10 [01:27<00:00,  9.22s/it]


,precision,recall,f1-score,support
score,,,,
blues,0.698,0.650,0.667,20.0
classical,0.878,0.915,0.895,20.0
country,0.613,0.595,0.602,20.0
hiphop,0.733,0.685,0.704,20.0
jazz,0.748,0.750,0.746,20.0
metal,0.733,0.805,0.764,20.0
pop,0.739,0.790,0.762,20.0
reggae,0.681,0.705,0.688,20.0
rock,0.508,0.420,0.450,20.0


In [81]:
check_model(grid_lrg, X9, y9, encoder9, 5)


  0%|          | 0/5 [00:00<?, ?it/s]

Fitting 3 folds for each of 5 candidates, totalling 15 fits












Process ForkPoolWorker-884:
Process ForkPoolWorker-881:
Process ForkPoolWorker-882:
Process ForkPoolWorker-883:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/anton/anaconda/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/Users/anton/anaconda/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/Users/anton/anaconda/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/anton/anaconda/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/Users/anton/anaconda/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/anton/anaconda/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/anton/anaconda/li

KeyboardInterrupt: 

In [25]:
check_model(grid_svc, X9, y9, encoder9, 10)

100%|██████████| 10/10 [01:50<00:00, 11.13s/it]


,precision,recall,f1-score,support
score,,,,
blues,0.746,0.800,0.769,20.0
classical,0.960,0.900,0.928,20.0
country,0.708,0.720,0.709,20.0
hiphop,0.729,0.715,0.720,20.0
jazz,0.857,0.820,0.831,20.0
metal,0.788,0.785,0.783,20.0
pop,0.792,0.770,0.777,20.0
reggae,0.724,0.755,0.734,20.0
rock,0.540,0.505,0.516,20.0


In [26]:
check_model(eclf, X9, y9, encoder9, 10)

100%|██████████| 10/10 [03:12<00:00, 18.35s/it]


,precision,recall,f1-score,support
score,,,,
blues,0.822,0.765,0.789,20.0
classical,0.928,0.925,0.923,20.0
country,0.767,0.725,0.742,20.0
hiphop,0.731,0.760,0.744,20.0
jazz,0.824,0.825,0.820,20.0
metal,0.781,0.815,0.797,20.0
pop,0.795,0.780,0.785,20.0
reggae,0.754,0.760,0.752,20.0
rock,0.527,0.515,0.515,20.0


In [ ]:
X_train_idx, X_test_idx, y_train_, y_test_ = train_test_split(np.arange(900), y9, test_size=0.2, random_state=5, stratify=y)

## Модель из 10 жанров

In [27]:
encoder10 = LabelEncoder()
y10 = encoder10.fit_transform(genre_list)

In [70]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y10, test_size=0.2, random_state=5, stratify=y10)
eclf.fit(X_train, y_train)
target = encoder10.inverse_transform(np.arange(10))
print(classification_report(y_test, eclf.predict(X_test), target_names=target))

Fitting 3 folds for each of 60 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:   13.0s finished


Fitting 3 folds for each of 9 candidates, totalling 27 fits


[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed:    7.6s finished


             precision    recall  f1-score   support

      blues       0.82      0.70      0.76        20
  classical       0.86      0.90      0.88        20
    country       0.62      0.65      0.63        20
      disco       0.48      0.50      0.49        20
     hiphop       0.93      0.70      0.80        20
       jazz       0.70      0.80      0.74        20
      metal       0.86      0.90      0.88        20
        pop       0.73      0.80      0.76        20
     reggae       0.72      0.65      0.68        20
       rock       0.43      0.45      0.44        20

avg / total       0.71      0.70      0.71       200



/Users/anton/anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/anton/anaconda/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [28]:
check_model(grid_rfc, X_scaled, y10, encoder10, 10)

100%|██████████| 10/10 [01:32<00:00,  8.98s/it]


,precision,recall,f1-score,support
score,,,,
blues,0.703,0.640,0.664,20.0
classical,0.878,0.910,0.892,20.0
country,0.597,0.590,0.589,20.0
disco,0.514,0.600,0.551,20.0
hiphop,0.697,0.555,0.615,20.0
jazz,0.734,0.775,0.753,20.0
metal,0.726,0.840,0.777,20.0
pop,0.671,0.790,0.722,20.0
reggae,0.586,0.680,0.627,20.0


In [29]:
check_model(grid_svc, X_scaled, y10, encoder10, 10)

100%|██████████| 10/10 [02:07<00:00, 12.60s/it]


,precision,recall,f1-score,support
score,,,,
blues,0.817,0.800,0.803,20.0
classical,0.922,0.915,0.917,20.0
country,0.702,0.720,0.706,20.0
disco,0.571,0.615,0.589,20.0
hiphop,0.701,0.715,0.703,20.0
jazz,0.869,0.820,0.840,20.0
metal,0.834,0.810,0.818,20.0
pop,0.744,0.785,0.761,20.0
reggae,0.683,0.700,0.688,20.0


In [30]:
check_model(eclf, X_scaled, y10, encoder10, 10)

100%|██████████| 10/10 [03:28<00:00, 21.12s/it]


,precision,recall,f1-score,support
score,,,,
blues,0.852,0.770,0.805,20.0
classical,0.898,0.905,0.900,20.0
country,0.744,0.720,0.724,20.0
disco,0.601,0.620,0.605,20.0
hiphop,0.734,0.725,0.722,20.0
jazz,0.853,0.815,0.831,20.0
metal,0.796,0.845,0.817,20.0
pop,0.744,0.800,0.768,20.0
reggae,0.667,0.725,0.692,20.0


## Средние параметры моделей

In [313]:
model_6

,precision,recall,f1-score,support
score,,,,
classical,0.873,0.940,0.906,30.0
metal,0.783,0.896,0.834,30.0
hiphop,0.741,0.838,0.786,30.0
pop,0.799,0.790,0.791,30.0
rock,0.765,0.614,0.678,30.0
jazz,0.928,0.774,0.843,30.0
avg / total,0.814,0.808,0.806,180.0


In [314]:
model_7

,precision,recall,f1-score,support
score,,,,
classical,0.880,0.933,0.905,30.0
metal,0.783,0.877,0.826,30.0
hiphop,0.700,0.791,0.742,30.0
pop,0.748,0.799,0.771,30.0
rock,0.691,0.608,0.643,30.0
reggae,0.762,0.637,0.692,30.0
jazz,0.902,0.785,0.836,30.0
avg / total,0.781,0.778,0.771,210.0


In [315]:
model_8

,precision,recall,f1-score,support
score,,,,
classical,0.878,0.934,0.905,30.0
metal,0.740,0.883,0.804,30.0
blues,0.656,0.813,0.725,30.0
hiphop,0.756,0.810,0.779,30.0
pop,0.758,0.779,0.765,30.0
rock,0.555,0.433,0.485,30.0
country,0.769,0.541,0.628,30.0
jazz,0.881,0.727,0.791,30.0
avg / total,0.748,0.739,0.736,240.0


In [316]:
model_9

,precision,recall,f1-score,support
score,,,,
classical,0.865,0.947,0.902,30.0
metal,0.737,0.859,0.791,30.0
blues,0.657,0.807,0.722,30.0
hiphop,0.684,0.754,0.712,30.0
pop,0.732,0.797,0.761,30.0
rock,0.525,0.445,0.482,30.0
country,0.727,0.547,0.621,30.0
reggae,0.710,0.600,0.650,30.0
jazz,0.895,0.702,0.784,30.0


In [317]:
model_10

,precision,recall,f1-score,support
score,,,,
classical,0.860,0.936,0.895,30.0
metal,0.708,0.877,0.783,30.0
blues,0.715,0.784,0.743,30.0
hiphop,0.613,0.694,0.649,30.0
disco,0.523,0.631,0.569,30.0
pop,0.706,0.749,0.725,30.0
rock,0.450,0.330,0.377,30.0
country,0.714,0.615,0.658,30.0
reggae,0.725,0.546,0.619,30.0
